In [ ]:
clauimport os

project_folder = "/content/drive/MyDrive/Colab_Notebooks/MDDdataset/symptom_sum_top16/claude_results"

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(project_folder) == False:
    os.mkdir(project_folder)
    print(project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(project_folder)
create_and_set_working_directory(project_folder)

In [ ]:
import pickle
with open('/content/drive/MyDrive/Colab_Notebooks/MDDdataset/symptom_sum_top16/test.pkl', 'rb') as f:
      raw_test_data = pickle.load(f)
with open('/content/drive/MyDrive/Colab_Notebooks/MDDdataset/symptom_sum_top16/val.pkl', 'rb') as f:
      raw_val_data = pickle.load(f)
with open('/content/drive/MyDrive/Colab_Notebooks/MDDdataset/symptom_sum_top16/train.pkl', 'rb') as f:
      raw_train_data = pickle.load(f)

In [ ]:
import pandas as pd
train_data=pd.DataFrame(raw_train_data)
test_data_diseases=pd.DataFrame(raw_test_data)
val_data=pd.DataFrame(raw_val_data)

In [ ]:
test_data=test_data_diseases[['id', 'selected_posts']]

In [ ]:
all_data = pd.concat([train_data, val_data], axis=0, ignore_index=True)
all_data = pd.concat([all_data, test_data_diseases], axis=0, ignore_index=True)

In [ ]:
y_true_train=[]
for row in all_data['diseases']:
  if 'adhd' in row:
    y_true_train.append(1)
  else:
    y_true_train.append(0)

In [ ]:
y_true_test=[]
for row in test_data_diseases['diseases']:
  if 'adhd' in row:
    y_true_test.append(1)
  else:
    y_true_test.append(0)

In [ ]:
y_train_best=np.load('y_train_best.npy')

### **Claude 3, all_data, test_data (non-assessment scale)**

In [ ]:
def preprocess_text(text, stop_words):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

In [ ]:
#baseline+test
import numpy as np
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
from sklearn.model_selection import cross_validate
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk

!python -m nltk.downloader stopwords
# Load the trained model and vectorizer
model = LogisticRegression(multi_class='multinomial', solver='saga')
vectorizer = TfidfVectorizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

# Load and preprocess the training data
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))

# Train the model
X = vectorizer.fit_transform(all_data['selected_posts'])
y = y_true_train


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
# Perform cross-validation
model.fit(X_train, y_train)
predictions=model.predict(X_test)

# Number of bootstrap samples
num_bootstraps = 1000

# Function to calculate F1, recall, and precision scores
def bootstrap_scores(true_labels, preds):
    indices = np.random.choice(len(true_labels), len(true_labels), replace=True)
    bootstrap_true_labels = np.take(true_labels, indices)
    bootstrap_preds = np.take(preds, indices)
    f1 = f1_score(bootstrap_true_labels, bootstrap_preds)
    recall = recall_score(bootstrap_true_labels, bootstrap_preds)
    precision = precision_score(bootstrap_true_labels, bootstrap_preds)
    return f1, recall, precision

# Generate bootstrap samples and calculate scores
bootstrap_scores_list = [bootstrap_scores(y_true_test, predictions) for _ in range(num_bootstraps)]

# Calculate standard deviation of scores
f1_scores, recall_scores, precision_scores = zip(*bootstrap_scores_list)
std_f1_score = np.std(f1_scores)
std_recall_score = np.std(recall_scores)
std_precision_score = np.std(precision_scores)

# Print results
print("Standard Deviation of F1 Scores (Bootstrap):", std_f1_score)
print("Standard Deviation of Recall Scores (Bootstrap):", std_recall_score)
print("Standard Deviation of Precision Scores (Bootstrap):", std_precision_score)

# Print results
print("f1_score", np.mean(f1_scores))
print("recall", np.mean(recall_scores))
print("precision", np.mean(precision_scores))
print("std_f1_score", std_f1_score)
print("std_recall_score", std_recall_score)
print("std_precision_score", std_precision_score)


/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Standard Deviation of F1 Scores (Bootstrap): 0.035015867824789694
Standard Deviation of Recall Scores (Bootstrap): 0.028756520982582138
Standard Deviation of Precision Scores (Bootstrap): 0.045588253142936214
f1_score 0.3902573661161953
recall 0.26028061006528447
precision 0.785713762226094
std_f1_score 0.035015867824789694
std_recall_score 0.028756520982582138
std_precision_score 0.045588253142936214


In [ ]:
with open('adhd-test-simple.npy', 'wb') as f:
    np.save(f, predictions)

In [ ]:
#baseline+all
import numpy as np
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
from sklearn.model_selection import cross_validate, train_test_split
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk
!python -m nltk.downloader stopwords


# Load the trained model and vectorizer
model = LogisticRegression(multi_class='multinomial', solver='saga')
vectorizer = TfidfVectorizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

# Function to calculate F1, recall, and precision scores
def bootstrap_scores(true_labels, preds):
    indices = np.random.choice(len(true_labels), len(true_labels), replace=True)
    bootstrap_true_labels = np.take(true_labels, indices)
    bootstrap_preds = np.take(preds, indices)
    f1 = f1_score(bootstrap_true_labels, bootstrap_preds)
    recall = recall_score(bootstrap_true_labels, bootstrap_preds)
    precision = precision_score(bootstrap_true_labels, bootstrap_preds)
    return f1, recall, precision


# Load and preprocess the training data
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))

# Preprocess the text data
X = vectorizer.fit_transform(all_data['selected_posts'])
y = y_true_train


# Define the scoring metrics
scoring = {
    'f1': make_scorer(f1_score),
    'sensitivity': make_scorer(recall_score),
    'specificity': make_scorer(lambda y_true, y_pred: precision_score(y_true, y_pred))
}

# Perform cross-validation
cv_scores = cross_validate(model, X, y, cv=5, scoring=scoring, return_estimator=True, return_indices=True)

# Calculate the mean and standard deviation of the scores
f1_mean = np.mean(cv_scores['test_f1'])
f1_std = np.std(cv_scores['test_f1'])
sensitivity_mean = np.mean(cv_scores['test_sensitivity'])
sensitivity_std = np.std(cv_scores['test_sensitivity'])
specificity_mean = np.mean(cv_scores['test_specificity'])
specificity_std = np.std(cv_scores['test_specificity'])
'''
# Get the predictions for each fold
fold_predictions_cross = []
for estimator, test_index in zip(cv_scores['estimator'], cv_scores['indices']['test']):
    print(test_index)
    X_test_fold = X_combined.tocsr()[test_index]
    y_test_fold = y[test_index]
    y_pred_fold = estimator.predict(X_test_fold)
    fold_predictions_cross.append((y_test_fold, y_pred_fold))
'''
print("Cross-validation scores:")
print(f"F1 Score: {f1_mean:.4f} ± {f1_std:.4f}")
print(f"Sensitivity: {sensitivity_mean:.4f} ± {sensitivity_std:.4f}")
print(f"Specificity: {specificity_mean:.4f} ± {specificity_std:.4f}")

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Cross-validation scores:
F1 Score: 0.3921 ± 0.1020
Sensitivity: 0.2881 ± 0.0473
Specificity: 0.7786 ± 0.3102


In [ ]:
with open('adhd-all-simple.npy', 'wb') as f:
    np.save(f, cv_scores)

### **ADHD**


In [ ]:
checklist_items = [
    # ATTENTION 314.00 (≥6/9)
    'Fails to give close attention to details, careless mistakes',
    'Difficulty sustaining attention in tasks or fun activities',
    'Does not seem to listen when spoken to directly',
    'Does not follow through on instructions and fails to finish work',
    'Difficulty organizing tasks and activities',
    'Avoids tasks that require sustained mental effort (boring)',
    'Losing things',
    'Easily distracted',
    'Forgetful in daily activities',

    # HYPERACTIVE/IMPULSIVE 314.01 (≥6/9)
    'Fidgety or squirms in seat',
    'Leaves seat when sitting is expected',
    'Feels restless',
    'Difficulty in doing fun things quietly',
    'Always on the go or acts as if "driven by a motor"',
    'Talks excessively',
    'Blurts answers before questions have been completed',
    'Difficulty awaiting turn',
    'Interrupting or intruding on others',

    # OPPOSITIONAL DEFIANT DISORDER 313.81 (>4/8)
    'Loses temper',
    'Argues with adults',
    'Actively defies or refuses to comply with requests or rules',
    'Deliberately annoys people',
    'Blames others for his or her mistakes or misbehavior',
    'Touchy or easily annoyed by others',
    'Angry or resentful',
    'Spiteful or vindictive'
]

In [ ]:
#baseline+test
import numpy as np
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
from sklearn.model_selection import cross_validate, train_test_split
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk
nltk.download('stopwords')

# Load the trained model and vectorizer
model = LogisticRegression(multi_class='multinomial', solver='saga')
vectorizer = TfidfVectorizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

# Function to calculate F1, recall, and precision scores
def bootstrap_scores(true_labels, preds):
    indices = np.random.choice(len(true_labels), len(true_labels), replace=True)
    bootstrap_true_labels = np.take(true_labels, indices)
    bootstrap_preds = np.take(preds, indices)
    f1 = f1_score(bootstrap_true_labels, bootstrap_preds, average='weighted')
    recall = recall_score(bootstrap_true_labels, bootstrap_preds, average='weighted')
    precision = precision_score(bootstrap_true_labels, bootstrap_preds, average='weighted')
    return f1, recall, precision

# Load and preprocess the training data
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))

# Preprocess the text data
X_text = vectorizer.fit_transform(all_data['selected_posts'])


X_ratings = []
for _, row in all_data.iterrows():
    user_posts = row['selected_posts']
    user_ratings = [int(item.lower() in user_posts.lower()) for item in checklist_items]
    X_ratings.append(user_ratings)

X_ratings = np.array(X_ratings)

# Combine text features and rating features
X_combined = np.hstack((X_text.toarray(), X_ratings))

y = y_true_train

# Define the scoring metrics
scoring = {
    'f1': make_scorer(f1_score, average='weighted'),
    'sensitivity': make_scorer(recall_score, average='weighted'),
    'specificity': make_scorer(lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted'))
}

# Perform cross-validation
cv_scores = cross_validate(model, X_combined, y, cv=5, scoring=scoring, return_estimator=True)

# Calculate the mean and standard deviation of the scores
f1_mean = np.mean(cv_scores['test_f1'])
f1_std = np.std(cv_scores['test_f1'])
sensitivity_mean = np.mean(cv_scores['test_sensitivity'])
sensitivity_std = np.std(cv_scores['test_sensitivity'])
specificity_mean = np.mean(cv_scores['test_specificity'])
specificity_std = np.std(cv_scores['test_specificity'])

print("Cross-validation scores:")
print(f"F1 Score: {f1_mean:.4f} ± {f1_std:.4f}")
print(f"Sensitivity: {sensitivity_mean:.4f} ± {sensitivity_std:.4f}")
print(f"Specificity: {specificity_mean:.4f} ± {specificity_std:.4f}")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#baseline+all
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk
!python -m nltk.downloader stopwords

def preprocess_text(text, stop_words):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

def predict_ratings(checklist_items, model, vectorizer, stop_words):
    # Preprocess the checklist items
    preprocessed_items = [preprocess_text(item, stop_words) for item in checklist_items]

    # Convert checklist items to TF-IDF vectors
    item_vectors = vectorizer.transform(preprocessed_items)

    # Predict ratings using the trained model
    predicted_ratings = model.predict(item_vectors)

    return predicted_ratings

def is_adhd_patient(checklist_items, model, vectorizer, stop_words):
    # Define the thresholds for each section
    section_thresholds = {
        'attention': 6,
        'hyperactive_impulsive': 6,
        'oppositional_defiant': 4
    }

    # Initialize variables to store the count of items with ratings 2 and 3
    section_counts = {
        'attention': 0,
        'hyperactive_impulsive': 0,
        'oppositional_defiant': 0
    }

    # Predict ratings for the checklist items
    predicted_ratings = predict_ratings(checklist_items, model, vectorizer, stop_words)

    # Iterate over each item and its predicted rating
    for item, rating in zip(checklist_items, predicted_ratings):
        # Determine the section of the item based on its name
        if 'ATTENTION' in item:
            section = 'attention'
        elif 'HYPERACTIVE/IMPULSIVE' in item:
            section = 'hyperactive_impulsive'
        else:
            section = 'oppositional_defiant'

        # Increment the count if the rating is 2 or 3
        if rating >= 2:
            section_counts[section] += 1

    # Check if any section exceeds its threshold
    for section, count in section_counts.items():
        if count >= section_thresholds[section]:
            return True

    return False

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import cross_validate
'''
# Preprocess the text data
stop_words = set(stopwords.words('english'))

all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))
'''
# Split the data into features (X) and target (y)

X = all_data['selected_posts']
y = y_true_train

# Convert the target variable to numerical labels
y = pd.factorize(y)[0]

# Convert text to Bag-of-Words representation
vectorizer = CountVectorizer()
X_vec = vectorizer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.1)

# Convert text to Bag-of-Words representation
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Define the scoring metrics
scoring = {
    'f1': make_scorer(f1_score, average='weighted'),
    'sensitivity': make_scorer(recall_score, average='weighted'),
    'specificity': make_scorer(lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted'))
}

# Calculate the mean and standard deviation of the scores
f1_mean = np.mean(cv_scores['test_f1'])
f1_std = np.std(cv_scores['test_f1'])
sensitivity_mean = np.mean(cv_scores['test_sensitivity'])
sensitivity_std = np.std(cv_scores['test_sensitivity'])
specificity_mean = np.mean(cv_scores['test_specificity'])
specificity_std = np.std(cv_scores['test_specificity'])
predictions_list = cv_scores['test_predictions']

print("Cross-validation scores:")
print(f"F1 Score: {f1_mean:.4f} ± {f1_std:.4f}")
print(f"Sensitivity: {sensitivity_mean:.4f} ± {sensitivity_std:.4f}")
print(f"Specificity: {specificity_mean:.4f} ± {specificity_std:.4f}")



Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/usr/lib/python3.10/runpy.py", line 110, in _get_module_details
    __import__(pkg_name)
  File "/usr/local/lib/python3.10/dist-packages/nltk/__init__.py", line 133, in <module>
    from nltk.collocations import *
  File "/usr/local/lib/python3.10/dist-packages/nltk/collocations.py", line 36, in <module>
    from nltk.metrics import (
  File "/usr/local/lib/python3.10/dist-packages/nltk/metrics/__init__.py", line 17, in <module>
    from nltk.metrics.aline import align
  File "/usr/local/lib/python3.10/dist-packages/nltk/metrics/aline.py", line 41, in <module>
    import numpy as np
  File "/usr/local/lib/python3.10/dist-packages/numpy/__init__.py", line 139, in <module>
    from . import core
  File "/usr/local/lib/python3.10/dist-packages/numpy/core/__init__.py", line 71, in <module>
    from . import n

NameError: name 'y_true_train' is not defined

In [ ]:
#assessment+ test
#!pip install -U scikit-learn
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import cross_validate
from scipy.sparse import hstack
!python -m nltk.downloader stopwords

def preprocess_text(text, stop_words):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

def generate_ratings(user_posts, assessment_items, vectorizer, stop_words):
    user_ratings = []
    for item in assessment_items:
        if any(item.lower() in post.lower() for post in user_posts):
            user_ratings.append(1)  # Item is present in user's posts
        else:
            user_ratings.append(0)  # Item is not present in user's posts
    return user_ratings

# Preprocess the text data
stop_words = set(stopwords.words('english'))

all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: [preprocess_text(post, stop_words) for post in x])


# Split the data into features (X) and target (y)
X_text = all_data['selected_posts']
y = y_true_train

# Convert the target variable to numerical labels
y = pd.factorize(y)[0]

np.random.seed(42)  # Set a fixed random seed
# Convert text to Bag-of-Words representation
vectorizer = CountVectorizer()
X_text_vec = vectorizer.fit_transform([' '.join(posts) for posts in X_text])

# Generate ratings for each data point
X_ratings = []
for _, row in all_data.iterrows():
    user_posts = row['selected_posts']
    user_ratings = generate_ratings(user_posts, checklist_items, vectorizer, stop_words)
    X_ratings.append(user_ratings)

# Convert ratings to a numpy array
X_ratings = np.array(X_ratings)

# Combine text features and rating features
X_combined = hstack([X_text_vec, X_ratings])

# Train the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

#train a model
X_combined=X_combined.tocsr()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

rf_classifier.fit(X_train, y_train)
predictions=rf_classifier.predict(X_test)

# Number of bootstrap samples
num_bootstraps = 100

# Function to calculate F1, recall, and precision scores
def bootstrap_scores(true_labels, preds):
    indices = np.random.choice(len(true_labels), len(true_labels), replace=True)
    bootstrap_true_labels = np.take(true_labels, indices)
    bootstrap_preds = np.take(preds, indices)
    f1 = f1_score(bootstrap_true_labels, bootstrap_preds)
    recall = recall_score(bootstrap_true_labels, bootstrap_preds)
    precision = precision_score(bootstrap_true_labels, bootstrap_preds)
    return f1, recall, precision

# Generate bootstrap samples and calculate scores
bootstrap_scores_list = [bootstrap_scores(y_test, predictions) for _ in range(num_bootstraps)]

# Calculate standard deviation of scores
f1_scores, recall_scores, precision_scores = zip(*bootstrap_scores_list)
std_f1_score = np.std(f1_scores)
std_recall_score = np.std(recall_scores)
std_precision_score = np.std(precision_scores)

# Print results
print("Standard Deviation of F1 Scores (Bootstrap):", std_f1_score)
print("Standard Deviation of Recall Scores (Bootstrap):", std_recall_score)
print("Standard Deviation of Precision Scores (Bootstrap):", std_precision_score)

# Print results
print("f1_score", np.mean(f1_scores))
print("recall", np.mean(recall_scores))
print("precision", np.mean(precision_scores))
print("std_f1_score", std_f1_score)
print("std_recall_score", std_recall_score)
print("std_precision_score", std_precision_score)

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Standard Deviation of F1 Scores (Bootstrap): 0.008695428863183368
Standard Deviation of Recall Scores (Bootstrap): 0.004453362373831768
Standard Deviation of Precision Scores (Bootstrap): 0.17063807140520212
f1_score 0.020997814812753948
recall 0.010649207581614917
precision 0.8532427849927849
std_f1_score 0.008695428863183368
std_recall_score 0.004453362373831768
std_precision_score 0.17063807140520212


In [ ]:
with open('adhd-test-assessment.npy', 'wb') as f:
    np.save(f, predictions)

In [ ]:
with open('adhd-test-assessment-Xtest.npy', 'wb') as f:
    np.save(f, X_test)

In [ ]:
#assessment+ all
!pip install -U scikit-learn
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import cross_validate
from scipy.sparse import hstack
!python -m nltk.downloader stopwords

def preprocess_text(text, stop_words):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

def generate_ratings(user_posts, assessment_items, vectorizer, stop_words):
    user_ratings = []
    for item in assessment_items:
        if any(item.lower() in post.lower() for post in user_posts):
            user_ratings.append(1)  # Item is present in user's posts
        else:
            user_ratings.append(0)  # Item is not present in user's posts
    return user_ratings

# Preprocess the text data
stop_words = set(stopwords.words('english'))
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: [preprocess_text(post, stop_words) for post in x])

# Split the data into features (X) and target (y)
X_text = all_data['selected_posts']
y = y_true_train

# Convert the target variable to numerical labels
y = pd.factorize(y)[0]

# Convert text to Bag-of-Words representation
vectorizer = CountVectorizer()
X_text_vec = vectorizer.fit_transform([' '.join(posts) for posts in X_text])

# Generate ratings for each data point
X_ratings = []
for _, row in all_data.iterrows():
    user_posts = row['selected_posts']
    user_ratings = generate_ratings(user_posts, checklist_items, vectorizer, stop_words)
    X_ratings.append(user_ratings)

# Convert ratings to a numpy array
X_ratings = np.array(X_ratings)

# Combine text features and rating features
X_combined = hstack([X_text_vec, X_ratings])

# Train the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Define the scoring metrics
scoring = {
    'f1': make_scorer(f1_score),
    'sensitivity': make_scorer(recall_score),
    'specificity': make_scorer(lambda y_true, y_pred: precision_score(y_true, y_pred))
}

# Perform cross-validation
cv_scores = cross_validate(rf_classifier, X_combined, y, cv=5, scoring=scoring, return_estimator=True, return_indices=True)

# Calculate the mean and standard deviation of the scores
f1_mean = np.mean(cv_scores['test_f1'])
f1_std = np.std(cv_scores['test_f1'])
sensitivity_mean = np.mean(cv_scores['test_sensitivity'])
sensitivity_std = np.std(cv_scores['test_sensitivity'])
specificity_mean = np.mean(cv_scores['test_specificity'])
specificity_std = np.std(cv_scores['test_specificity'])
'''
# Get the predictions for each fold
fold_predictions_cross = []
for estimator, test_index in zip(cv_scores['estimator'], cv_scores['indices']['test']):
    print(test_index)
    X_test_fold = X_combined.tocsr()[test_index]
    y_test_fold = y[test_index]
    y_pred_fold = estimator.predict(X_test_fold)
    fold_predictions_cross.append((y_test_fold, y_pred_fold))
'''
print("Cross-validation scores:")
print(f"F1 Score: {f1_mean:.4f} ± {f1_std:.4f}")
print(f"Sensitivity: {sensitivity_mean:.4f} ± {sensitivity_std:.4f}")
print(f"Specificity: {specificity_mean:.4f} ± {specificity_std:.4f}")


/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Cross-validation scores:
F1 Score: 0.0259 ± 0.0355
Sensitivity: 0.0156 ± 0.0230
Specificity: 0.5043 ± 0.3654


In [ ]:
# Get the predictions for each fold
fold_predictions_cross = []
for estimator, test_index in zip(cv_scores['estimator'], cv_scores['indices']['test']):
    print(test_index)
    X_test_fold = X_combined.tocsr()[test_index]
    y_test_fold = y[test_index]
    y_pred_fold = estimator.predict(X_test_fold)
    fold_predictions_cross.append((y_test_fold, y_pred_fold))

print("Cross-validation scores:")
print(f"F1 Score: {f1_mean:.4f} ± {f1_std:.4f}")
print(f"Sensitivity: {sensitivity_mean:.4f} ± {sensitivity_std:.4f}")
print(f"Specificity: {specificity_mean:.4f} ± {specificity_std:.4f}")

[   0    1    2 ... 6866 6868 6869]
[ 3106  3110  3113 ... 11813 11814 11815]
[ 4210  4211  4214 ... 16659 16660 16661]
[ 5322  5323  5326 ... 21505 21506 21507]
[ 6552  6553  6554 ... 26602 26603 26604]
Cross-validation scores:
F1 Score: 0.8682 ± 0.0030
Sensitivity: 0.9055 ± 0.0113
Specificity: 0.8862 ± 0.0247


In [ ]:
with open('adhd-all-assessment.npy', 'wb') as f:
    np.save(f, cv_scores)

### **Depression**

In [ ]:
assessment_items = [
    'Feeling sad or down in the dumps',
    'Feeling unhappy or blue',
    'Crying spells or tearfulness',
    'Feeling discouraged',
    'Feeling hopeless',
    'Low self-esteem',
    'Feeling worthless or inadequate',
    'Guilt or shame',
    'Criticizing yourself or blaming others',
    'Difficulty making decisions',
    'Loss of interest in family, friends or colleagues',
    'Loneliness',
    'Spending less time with family or friends',
    'Loss of motivation',
    'Loss of interest in work or other activities',
    'Avoiding work or other activities',
    'Loss of pleasure or satisfaction in life',
    'Feeling tired',
    'Difficulty sleeping or sleeping too much',
    'Decreased or increased appetite',
    'Loss of interest in sex',
    'Worrying about your health',
    'Do you have any suicidal thoughts?',
    'Would you like to end your life?',
    'Do you have a plan for harming yourself?'
]

In [ ]:
#assessment+all
!pip install -U scikit-learn
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import cross_validate
from scipy.sparse import hstack
!python -m nltk.downloader stopwords

def preprocess_text(text, stop_words):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

def generate_ratings(user_posts, assessment_items, vectorizer, stop_words):
    user_ratings = []
    for item in assessment_items:
        if any(item.lower() in post.lower() for post in user_posts):
            user_ratings.append(4)  # Item is present in user's posts, assign the highest rating
        else:
            user_ratings.append(0)  # Item is not present in user's posts, assign the lowest rating
    return user_ratings

# Preprocess the text data
stop_words = set(stopwords.words('english'))

all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: [preprocess_text(post, stop_words) for post in x])

# Split the data into features (X) and target (y)
X_text = all_data['selected_posts']
y = y_true_train

# Convert the target variable to numerical labels
y = pd.factorize(y)[0]

# Convert text to Bag-of-Words representation
vectorizer = CountVectorizer()
X_text_vec = vectorizer.fit_transform([' '.join(posts) for posts in X_text])

# Generate ratings for each data point
X_ratings = []
for _, row in all_data.iterrows():
    user_posts = row['selected_posts']
    user_ratings = generate_ratings(user_posts, checklist_items, vectorizer, stop_words)
    X_ratings.append(user_ratings)

# Convert ratings to a numpy array
X_ratings = np.array(X_ratings)

# Combine text features and rating features
X_combined = hstack([X_text_vec, X_ratings])

# Train the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100)

# Define the scoring metrics
scoring = {
    'f1': make_scorer(f1_score),
    'sensitivity': make_scorer(recall_score),
    'specificity': make_scorer(lambda y_true, y_pred: precision_score(y_true, y_pred))
}

# Perform cross-validation
cv_scores = cross_validate(rf_classifier, X_combined, y, cv=5, scoring=scoring, return_estimator=True, return_indices=True)

# Calculate the mean and standard deviation of the scores
f1_mean = np.mean(cv_scores['test_f1'])
f1_std = np.std(cv_scores['test_f1'])
sensitivity_mean = np.mean(cv_scores['test_sensitivity'])
sensitivity_std = np.std(cv_scores['test_sensitivity'])
specificity_mean = np.mean(cv_scores['test_specificity'])
specificity_std = np.std(cv_scores['test_specificity'])
'''
# Get the predictions for each fold
fold_predictions_cross = []
for estimator, test_index in zip(cv_scores['estimator'], cv_scores['indices']['test']):
    print(test_index)
    X_test_fold = X_combined.tocsr()[test_index]
    y_test_fold = y[test_index]
    y_pred_fold = estimator.predict(X_test_fold)
    fold_predictions_cross.append((y_test_fold, y_pred_fold))
'''
print("Cross-validation scores:")
print(f"F1 Score: {f1_mean:.4f} ± {f1_std:.4f}")
print(f"Sensitivity: {sensitivity_mean:.4f} ± {sensitivity_std:.4f}")
print(f"Specificity: {specificity_mean:.4f} ± {specificity_std:.4f}")



ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/usr/lib/python3.10/runpy.py", line 110, in _get_module_details
    __import__(pkg_name)
  File "/usr/local/lib/python3.10/dist-packages/nltk/__init__.py", line 133, in <module>
    from nltk.collocations import *
  File "/usr/local/lib/python3.10/dist-packages/nltk/collocations.py", line 36, in <module>
    from nltk.metrics import (
  File "/usr/local/lib/python3.10/dist-packages/nltk/metrics/__init__.py", line 38, in <module>
    from nltk.metrics.scores import (
  File "/usr/local/lib/python3.10/dist-packages/nltk/metrics/scores.py", line 19, in <module>
    from nltk.util import LazyConcatenation, LazyMap
  File "/usr/local/lib/python3.10/dist-packages/nltk/util.py", line 19, in <module>
    from urllib.request import (
  File "/usr/lib/python3.10/urllib/request.py"

KeyboardInterrupt: 

In [ ]:
with open('depression-all-assessment.npy', 'wb') as f:
    np.save(f, cv_scores)

In [ ]:
#baseline+test
import numpy as np
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
from sklearn.model_selection import cross_validate, train_test_split
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk
!python -m nltk.downloader stopwords

# Load the trained model and vectorizer
model = LogisticRegression(multi_class='multinomial', solver='saga')
vectorizer = TfidfVectorizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

# Function to calculate F1, recall, and precision scores
def bootstrap_scores(true_labels, preds):
    indices = np.random.choice(len(true_labels), len(true_labels), replace=True)
    bootstrap_true_labels = np.take(true_labels, indices)
    bootstrap_preds = np.take(preds, indices)
    f1 = f1_score(bootstrap_true_labels, bootstrap_preds)
    recall = recall_score(bootstrap_true_labels, bootstrap_preds)
    precision = precision_score(bootstrap_true_labels, bootstrap_preds)
    return f1, recall, precision


# Load and preprocess the training data
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))

# Preprocess the text data
X = vectorizer.fit_transform(all_data['selected_posts'])
y = y_true_train

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Number of bootstrap samples
num_bootstraps = 1000

# Generate bootstrap samples and calculate scores
bootstrap_scores_list = [bootstrap_scores(y_test, y_pred) for _ in range(num_bootstraps)]

# Calculate standard deviation of scores
f1_scores, recall_scores, precision_scores = zip(*bootstrap_scores_list)
std_f1_score = np.std(f1_scores)
std_recall_score = np.std(recall_scores)
std_precision_score = np.std(precision_scores)

# Print results
print("Mean F1 Score:", np.mean(f1_scores))
print("Mean Recall Score:", np.mean(recall_scores))
print("Mean Precision Score:", np.mean(precision_scores))
print("Standard Deviation of F1 Scores (Bootstrap):", std_f1_score)
print("Standard Deviation of Recall Scores (Bootstrap):", std_recall_score)
print("Standard Deviation of Precision Scores (Bootstrap):", std_precision_score)

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


KeyboardInterrupt: 

In [ ]:
with open('depression-test-simple.npy', 'wb') as f:
    np.save(f, y_pred)

In [ ]:
#baseline-all
import numpy as np
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
from sklearn.model_selection import cross_validate, train_test_split
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk
!python -m nltk.downloader stopwords


# Load the trained model and vectorizer
model = LogisticRegression(multi_class='multinomial', solver='saga')
vectorizer = TfidfVectorizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

# Function to calculate F1, recall, and precision scores
def bootstrap_scores(true_labels, preds):
    indices = np.random.choice(len(true_labels), len(true_labels), replace=True)
    bootstrap_true_labels = np.take(true_labels, indices)
    bootstrap_preds = np.take(preds, indices)
    f1 = f1_score(bootstrap_true_labels, bootstrap_preds)
    recall = recall_score(bootstrap_true_labels, bootstrap_preds)
    precision = precision_score(bootstrap_true_labels, bootstrap_preds)
    return f1, recall, precision

# Load and preprocess the training data
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))

# Preprocess the text data
X = vectorizer.fit_transform(all_data['selected_posts'])
y = y_true_train


# Define the scoring metrics
scoring = {
    'f1': make_scorer(f1_score),
    'sensitivity': make_scorer(recall_score),
    'specificity': make_scorer(lambda y_true, y_pred: precision_score(y_true, y_pred))
}

# Perform cross-validation
cv_scores = cross_validate(model, X, y, cv=5, scoring=scoring, return_estimator=True, return_indices=True)

# Calculate the mean and standard deviation of the scores
f1_mean = np.mean(cv_scores['test_f1'])
f1_std = np.std(cv_scores['test_f1'])
sensitivity_mean = np.mean(cv_scores['test_sensitivity'])
sensitivity_std = np.std(cv_scores['test_sensitivity'])
specificity_mean = np.mean(cv_scores['test_specificity'])
specificity_std = np.std(cv_scores['test_specificity'])
'''
# Get the predictions for each fold
fold_predictions_cross = []
for estimator, test_index in zip(cv_scores['estimator'], cv_scores['indices']['test']):
    print(test_index)
    X_test_fold = X.tocsr()[test_index]
    y_test_fold = y[test_index]
    y_pred_fold = estimator.predict(X_test_fold)
    fold_predictions_cross.append((y_test_fold, y_pred_fold))
'''
print("Cross-validation scores:")
print(f"F1 Score: {f1_mean:.4f} ± {f1_std:.4f}")
print(f"Sensitivity: {sensitivity_mean:.4f} ± {sensitivity_std:.4f}")
print(f"Specificity: {specificity_mean:.4f} ± {specificity_std:.4f}")


/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[   0    1    2 ... 7179 7180 7181]


TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
print("Cross-validation scores:")
print(f"F1 Score: {f1_mean:.4f} ± {f1_std:.4f}")
print(f"Sensitivity: {sensitivity_mean:.4f} ± {sensitivity_std:.4f}")
print(f"Specificity: {specificity_mean:.4f} ± {specificity_std:.4f}")

Cross-validation scores:
F1 Score: 0.1336 ± 0.0741
Sensitivity: 0.0853 ± 0.0676
Specificity: 0.8042 ± 0.2291


In [ ]:
with open('depression-all-simple.npy', 'wb') as f:
    np.save(f, cv_scores)

In [ ]:
#assessment+test
#!pip install -U scikit-learn
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import cross_validate
from scipy.sparse import hstack
!python -m nltk.downloader stopwords

def preprocess_text(text, stop_words):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

def generate_ratings(user_posts, assessment_items, vectorizer, stop_words):
    user_ratings = []
    for item in assessment_items:
        if any(item.lower() in post.lower() for post in user_posts):
            user_ratings.append(1)  # Item is present in user's posts
        else:
            user_ratings.append(0)  # Item is not present in user's posts
    return user_ratings

# Preprocess the text data
stop_words = set(stopwords.words('english'))

all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: [preprocess_text(post, stop_words) for post in x])

# Split the data into features (X) and target (y)
X_text = all_data['selected_posts']
y = y_true_train

# Convert the target variable to numerical labels
y = pd.factorize(y)[0]

# Convert text to Bag-of-Words representation
vectorizer = CountVectorizer()
X_text_vec = vectorizer.fit_transform([' '.join(posts) for posts in X_text])

# Generate ratings for each data point
X_ratings = []
for _, row in all_data.iterrows():
    user_posts = row['selected_posts']
    user_ratings = generate_ratings(user_posts, checklist_items, vectorizer, stop_words)
    X_ratings.append(user_ratings)

# Convert ratings to a numpy array
X_ratings = np.array(X_ratings)

# Combine text features and rating features
X_combined = hstack([X_text_vec, X_ratings])

# Train the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

X_combined=X_combined.tocsr()
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

#train a model
rf_classifier.fit(X_train, y_train)
predictions=rf_classifier.predict(X_test)

# Number of bootstrap samples
num_bootstraps = 1000

# Function to calculate F1, recall, and precision scores
def bootstrap_scores(true_labels, preds):
    indices = np.random.choice(len(true_labels), len(true_labels), replace=True)
    bootstrap_true_labels = np.take(true_labels, indices)
    bootstrap_preds = np.take(preds, indices)
    f1 = f1_score(bootstrap_true_labels, bootstrap_preds)
    recall = recall_score(bootstrap_true_labels, bootstrap_preds)
    precision = precision_score(bootstrap_true_labels, bootstrap_preds)
    return f1, recall, precision

# Generate bootstrap samples and calculate scores
bootstrap_scores_list = [bootstrap_scores(y_test, predictions) for _ in range(num_bootstraps)]

# Calculate standard deviation of scores
f1_scores, recall_scores, precision_scores = zip(*bootstrap_scores_list)
std_f1_score = np.std(f1_scores)
std_recall_score = np.std(recall_scores)
std_precision_score = np.std(precision_scores)

# Print results
print("Standard Deviation of F1 Scores (Bootstrap):", std_f1_score)
print("Standard Deviation of Recall Scores (Bootstrap):", std_recall_score)
print("Standard Deviation of Precision Scores (Bootstrap):", std_precision_score)

# Print results
print("f1_score", np.mean(f1_scores))
print("recall", np.mean(recall_scores))
print("precision", np.mean(precision_scores))
print("std_f1_score", std_f1_score)
print("std_recall_score", std_recall_score)
print("std_precision_score", std_precision_score)


/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Standard Deviation of F1 Scores (Bootstrap): 0.002403522959625154
Standard Deviation of Recall Scores (Bootstrap): 0.0009762779177247866
Standard Deviation of Precision Scores (Bootstrap): 0.004221790363950144
f1_score 0.941142753545742
recall 0.995489533463551
precision 0.8924324046102047
std_f1_score 0.002403522959625154
std_recall_score 0.0009762779177247866
std_precision_score 0.004221790363950144


In [ ]:
results=np.load('depression-test-assessment.npy', allow_pickle=True)

In [ ]:
from scipy.sparse import load_npz
X_text_vec= load_npz('vectorized_selected_posts.npz')

In [ ]:
X_test_best=load_npz('X_test_best.npz')

In [ ]:
'''
# Generate ratings for each data point
X_ratings = []
for _, row in all_data.iterrows():
    user_posts = row['selected_posts']
    user_ratings = generate_ratings(user_posts, checklist_items, vectorizer, stop_words)
    X_ratings.append(user_ratings)

# Convert ratings to a numpy array
X_ratings = np.array(X_ratings)

# Combine text features and rating features
X_combined = hstack([X_text_vec, X_ratings])

# Train the Random Forest Classifier
rf_classifier_depression = RandomForestClassifier(n_estimators=100, random_state=42)

X_combined=X_combined.tocsr()
'''
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_text_vec, y, test_size=0.2, random_state=42)

#train a model
rf_classifier_depression.fit(X_train, y_train)
predictions=rf_classifier_depression.predict(X_test)

# Number of bootstrap samples
num_bootstraps = 1000

# Function to calculate F1, recall, and precision scores
def bootstrap_scores(true_labels, preds):
    indices = np.random.choice(len(true_labels), len(true_labels), replace=True)
    bootstrap_true_labels = np.take(true_labels, indices)
    bootstrap_preds = np.take(preds, indices)
    f1 = f1_score(bootstrap_true_labels, bootstrap_preds)
    recall = recall_score(bootstrap_true_labels, bootstrap_preds)
    precision = precision_score(bootstrap_true_labels, bootstrap_preds)
    return f1, recall, precision

# Generate bootstrap samples and calculate scores
bootstrap_scores_list = [bootstrap_scores(y_test, predictions) for _ in range(num_bootstraps)]

# Calculate standard deviation of scores
f1_scores, recall_scores, precision_scores = zip(*bootstrap_scores_list)
std_f1_score = np.std(f1_scores)
std_recall_score = np.std(recall_scores)
std_precision_score = np.std(precision_scores)

# Print results
print("Standard Deviation of F1 Scores (Bootstrap):", std_f1_score)
print("Standard Deviation of Recall Scores (Bootstrap):", std_recall_score)
print("Standard Deviation of Precision Scores (Bootstrap):", std_precision_score)

# Print results
print("f1_score", np.mean(f1_scores))
print("recall", np.mean(recall_scores))
print("precision", np.mean(precision_scores))
print("std_f1_score", std_f1_score)
print("std_recall_score", std_recall_score)
print("std_precision_score", std_precision_score)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p

Standard Deviation of F1 Scores (Bootstrap): 0.008765145552053693
Standard Deviation of Recall Scores (Bootstrap): 0.004480297161621923
Standard Deviation of Precision Scores (Bootstrap): 0.063118935352238
f1_score 0.019972680355180905
recall 0.010106905999356363
precision 0.996
std_f1_score 0.008765145552053693
std_recall_score 0.004480297161621923
std_precision_score 0.063118935352238


In [ ]:
#assessment+test
#!pip install -U scikit-learn
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import cross_validate
from scipy.sparse import hstack
!python -m nltk.downloader stopwords

def preprocess_text(text, stop_words):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

def generate_ratings(user_posts, assessment_items, vectorizer, stop_words):
    user_ratings = []
    for item in assessment_items:
        if any(item.lower() in post.lower() for post in user_posts):
            user_ratings.append(1)  # Item is present in user's posts
        else:
            user_ratings.append(0)  # Item is not present in user's posts
    return user_ratings

# Preprocess the text data
stop_words = set(stopwords.words('english'))

all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: [preprocess_text(post, stop_words) for post in x])

# Split the data into features (X) and target (y)
X_text = all_data['selected_posts']
y = y_true_train

# Convert the target variable to numerical labels
y = pd.factorize(y)[0]

# Convert text to Bag-of-Words representation
vectorizer = CountVectorizer()
X_text_vec = vectorizer.fit_transform([' '.join(posts) for posts in X_text])

# Generate ratings for each data point
X_ratings = []
for _, row in all_data.iterrows():
    user_posts = row['selected_posts']
    user_ratings = generate_ratings(user_posts, checklist_items, vectorizer, stop_words)
    X_ratings.append(user_ratings)

# Convert ratings to a numpy array
X_ratings = np.array(X_ratings)

# Combine text features and rating features
X_combined = hstack([X_text_vec, X_ratings])


/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Get the predictions for each fold
fold_predictions_cross = []
for estimator, test_index in zip(cv_scores['estimator'], cv_scores['indices']['test']):
    print(test_index)
    X_test_fold = X_combined[test_index]
    y_test_fold = y[test_index]
    y_pred_fold = estimator.predict(X_test_fold)
    print(f1_score(y_test_fold, y_pred_fold))
    fold_predictions_cross.append((y_test_fold, y_pred_fold))

[   0    1    2 ... 7179 7180 7181]


ValueError: X has 506773 features, but RandomForestClassifier is expecting 506798 features as input.

In [ ]:
cv_scores

{'fit_time': array([217.37986803, 219.31158423, 231.4357605 , 224.59657741,
        230.65932393]),
 'score_time': array([5.11738658, 4.92695498, 4.51113391, 4.62715149, 4.2926929 ]),
 'estimator': [RandomForestClassifier(random_state=42),
  RandomForestClassifier(random_state=42),
  RandomForestClassifier(random_state=42),
  RandomForestClassifier(random_state=42),
  RandomForestClassifier(random_state=42)],
 'indices': {'train': (array([ 1182,  1183,  1184, ..., 26602, 26603, 26604]),
   array([    0,     1,     2, ..., 26602, 26603, 26604]),
   array([    0,     1,     2, ..., 26602, 26603, 26604]),
   array([    0,     1,     2, ..., 26602, 26603, 26604]),
   array([    0,     1,     2, ..., 21281, 21303, 21304])),
  'test': (array([   0,    1,    2, ..., 7179, 7180, 7181]),
   array([ 1182,  1183,  1184, ..., 11879, 11880, 11881]),
   array([ 2283,  2285,  2286, ..., 16579, 16580, 16581]),
   array([ 3380,  3382,  3384, ..., 21281, 21303, 21304]),
   array([21282, 21283, 21284, ..

In [ ]:
# Load the Random Forest model
import joblib
rf_model = joblib.load('random_forest_classifier.joblib')

In [ ]:
y_test_best=np.load('y_test_best.npy')

In [ ]:
predictions_test=rf_model.predict(X_test_best)

In [ ]:
f1_score(y_test_best, predictions_test)

0.9566241413150147

### **Autism**

In [ ]:
autism_assessment_items = {
    1: "S/he prefers to do things with others rather than on her/his own.",
    2: "S/he prefers to do things the same way over and over again.",
    3: "If s/he tries to imagine something, s/he finds it very easy to create a picture in her/his mind.",
    4: "S/he frequently gets so strongly absorbed in one thing that s/he loses sight of other things.",
    5: "S/he often notices small sounds when others do not.",
    6: "S/he usually notices car number plates or similar strings of information.",
    7: "Other people frequently tell her/him that what s/he has said is impolite, even though s/he thinks it is polite.",
    8: "When s/he is reading a story, s/he can easily imagine what the characters might look like.",
    9: "S/he is fascinated by dates.",
    10: "In a social group, s/he can easily keep track of several different people's conversations.",
    11: "S/he finds social situations easy.",
    12: "S/he tends to notice details that others do not.",
    13: "S/he would rather go to a library than a party.",
    14: "S/he finds making up stories easy.",
    15: "S/he finds her/himself drawn more strongly to people than to things.",
    16: "S/he tends to have very strong interests, which s/he gets upset about if s/he can't pursue.",
    17: "S/he enjoys social chit-chat.",
    18:"When s/he talks, it isn't always easy for others to get a word in edgeways.",
    19:"S/he is fascinated by numbers.",
    20:"When s/he is reading a story, s/he finds it difficult to work out the characters' intentions.",
    21:"S/he doesn't particularly enjoy reading fiction.",
    22:"S/he finds it hard to make new friends.",
    23:"S/he notices patterns in things all the time.",
    24:"S/he would rather go to the theatre than a museum.",
    25:"It does not upset him/her if his/her daily routine is disturbed.",
    26:"S/he frequently finds that s/he doesn't know how to keep a conversation going.",
    27:"S/he finds it easy to ''read between the lines'' when someone is talking to her/him.",
    28:"S/he usually concentrates more on the whole picture, rather than the small details.",
    29:"S/he is not very good at remembering phone numbers.",
    30:"S/he doesn't usually notice small changes in a situation, or a person's appearance.",
    31: "S/he knows how to tell if someone listening to him/her is getting bored.",
    32:"S/he finds it easy to do more than one thing at once.",
    33:"When s/he talks on the phone, s/he is not sure when it's her/his turn to speak.",
    34:"S/he enjoys doing things spontaneously.",
    35:"S/he is often the last to understand the point of a joke.",
    36:"S/he finds it easy to work out what someone is thinking or feeling just by looking at their face.",
    37:"If there is an interruption, s/he can switch back to what s/he was doing very quickly.",
    38:"S/he is good at social chit-chat.",
    39:"People often tell her/him that s/he keeps going on and on about the same thing.",
    40:"When s/he was younger, s/he used to enjoy playing games involving pretending with other children.",
    41:"S/he likes to collect information about categories of things (e.g. types of car, types of bird, types of train, types of plant, etc.).",
    42:"S/he finds it difficult to imagine what it would be like to be someone else.",
    43:"S/he likes to plan any activities s/he participates in carefully.",
    44:"S/he enjoys social occasions.",
    45:"S/he finds it difficult to work out people's intentions.",
    46:"New situations make him/her anxious.",
    47:"S/he enjoys meeting new people.",
    48:"S/he is a good diplomat.",
    49:"S/he is not very good at remembering people's date of birth.",
    50:"S/he finds it very easy to play games with children that involve pretending."}


In [ ]:
#assessment+all
!pip install -U scikit-learn
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import cross_validate
from scipy.sparse import hstack

!python -m nltk.downloader stopwords

def preprocess_text(text, stop_words):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

def generate_ratings(user_posts, autism_assessment_items, vectorizer, stop_words):
    user_ratings = []
    for item_number, item in autism_assessment_items.items():
        if any(item.lower() in post.lower() for post in user_posts):
            if item_number in [2, 4, 5, 6, 7, 9, 12, 13, 16, 18, 19, 20, 21, 22, 23, 26, 33, 35, 39, 41, 42, 43, 45, 46]:
                user_ratings.append(1)  # 'Definitely agree' or 'slightly agree' responses scored 1 point
            else:
                user_ratings.append(0)  # 'Definitely disagree' or 'slightly disagree' responses scored 0 points
        else:
            if item_number in [1, 3, 8, 10, 11, 14, 15, 17, 24, 25, 27, 28, 29, 30, 31, 32, 34, 36, 37, 38, 40, 44, 47, 48, 49, 50]:
                user_ratings.append(1)  # 'Definitely disagree' or 'slightly disagree' responses scored 1 point
            else:
                user_ratings.append(0)  # 'Definitely agree' or 'slightly agree' responses scored 0 points
    return user_ratings

# Preprocess the text data
stop_words = set(stopwords.words('english'))
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: [preprocess_text(post, stop_words) for post in x])

# Split the data into features (X) and target (y)
X_text = all_data['selected_posts']
y = y_true_train

# Convert the target variable to numerical labels
y = pd.factorize(y)[0]

# Convert text to Bag-of-Words representation
vectorizer = CountVectorizer()
X_text_vec = vectorizer.fit_transform([' '.join(posts) for posts in X_text])

# Generate ratings for each data point
X_ratings = []
for _, row in all_data.iterrows():
    user_posts = row['selected_posts']
    user_ratings = generate_ratings(user_posts, autism_assessment_items, vectorizer, stop_words)
    X_ratings.append(user_ratings)

# Convert ratings to a numpy array
X_ratings = np.array(X_ratings)

# Combine text features and rating features
X_combined = hstack([X_text_vec, X_ratings])

# Train the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=200, max_depth=10, min_samples_split=5, random_state=42)

# Define the scoring metrics
scoring = {
    'f1': make_scorer(f1_score),
    'sensitivity': make_scorer(recall_score),
    'specificity': make_scorer(lambda y_true, y_pred: precision_score(y_true, y_pred))
}

# Perform cross-validation
cv_scores = cross_validate(rf_classifier, X_combined, y, cv=5, scoring=scoring, return_estimator=True, return_indices=True)

# Calculate the mean and standard deviation of the scores
f1_mean = np.mean(cv_scores['test_f1'])
f1_std = np.std(cv_scores['test_f1'])
sensitivity_mean = np.mean(cv_scores['test_sensitivity'])
sensitivity_std = np.std(cv_scores['test_sensitivity'])
specificity_mean = np.mean(cv_scores['test_specificity'])
specificity_std = np.std(cv_scores['test_specificity'])
'''
# Get the predictions for each fold
fold_predictions_cross = []
for estimator, test_index in zip(cv_scores['estimator'], cv_scores['indices']['test']):
    print(test_index)
    X_test_fold = X_combined.tocsr()[test_index]
    y_test_fold = y[test_index]
    y_pred_fold = estimator.predict(X_test_fold)
    fold_predictions_cross.append((y_test_fold, y_pred_fold))
'''
print("Cross-validation scores:")
print(f"F1 Score: {f1_mean:.4f} ± {f1_std:.4f}")
print(f"Sensitivity: {sensitivity_mean:.4f} ± {sensitivity_std:.4f}")
print(f"Specificity: {specificity_mean:.4f} ± {specificity_std:.4f}")

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Cross-validation scores:
F1 Score: 0.0055 ± 0.0068
Sensitivity: 0.0028 ± 0.0034
Specificity: 0.4000 ± 0.4899


In [ ]:
with open('autism-all-assessment.npy', 'wb') as f:
    np.save(f, cv_scores)

In [ ]:
#assessment+test
!pip install -U scikit-learn
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import cross_validate
from scipy.sparse import hstack

!python -m nltk.downloader stopwords

def preprocess_text(text, stop_words):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

def generate_ratings(user_posts, autism_assessment_items, vectorizer, stop_words):
    user_ratings = []
    for item_number, item in autism_assessment_items.items():
        if any(item.lower() in post.lower() for post in user_posts):
            if item_number in [2, 4, 5, 6, 7, 9, 12, 13, 16, 18, 19, 20, 21, 22, 23, 26, 33, 35, 39, 41, 42, 43, 45, 46]:
                user_ratings.append(1)  # 'Definitely agree' or 'slightly agree' responses scored 1 point
            else:
                user_ratings.append(0)  # 'Definitely disagree' or 'slightly disagree' responses scored 0 points
        else:
            if item_number in [1, 3, 8, 10, 11, 14, 15, 17, 24, 25, 27, 28, 29, 30, 31, 32, 34, 36, 37, 38, 40, 44, 47, 48, 49, 50]:
                user_ratings.append(1)  # 'Definitely disagree' or 'slightly disagree' responses scored 1 point
            else:
                user_ratings.append(0)  # 'Definitely agree' or 'slightly agree' responses scored 0 points
    return user_ratings

# Preprocess the text data
stop_words = set(stopwords.words('english'))
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: [preprocess_text(post, stop_words) for post in x])

# Split the data into features (X) and target (y)
X_text = all_data['selected_posts']
y = y_true_train

# Convert the target variable to numerical labels
y = pd.factorize(y)[0]

# Convert text to Bag-of-Words representation
vectorizer = CountVectorizer()
X_text_vec = vectorizer.fit_transform([' '.join(posts) for posts in X_text])

# Generate ratings for each data point
X_ratings = []
for _, row in all_data.iterrows():
    user_posts = row['selected_posts']
    user_ratings = generate_ratings(user_posts, autism_assessment_items, vectorizer, stop_words)
    X_ratings.append(user_ratings)

# Convert ratings to a numpy array
X_ratings = np.array(X_ratings)

# Combine text features and rating features
X_combined = hstack([X_text_vec, X_ratings])

# Train the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=200, max_depth=10, min_samples_split=5)
X_combined = X_combined.tocsr()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.1)

# Train a model
rf_classifier.fit(X_train, y_train)
predictions = rf_classifier.predict(X_test)

# Number of bootstrap samples
num_bootstraps = 100

# Function to calculate F1, recall, and precision scores
def bootstrap_scores(true_labels, preds):
    indices = np.random.choice(len(true_labels), len(true_labels), replace=True)
    bootstrap_true_labels = np.take(true_labels, indices)
    bootstrap_preds = np.take(preds, indices)
    f1 = f1_score(bootstrap_true_labels, bootstrap_preds)
    recall = recall_score(bootstrap_true_labels, bootstrap_preds)
    precision = precision_score(bootstrap_true_labels, bootstrap_preds)
    return f1, recall, precision

# Generate bootstrap samples and calculate scores
bootstrap_scores_list = [bootstrap_scores(y_test, predictions) for _ in range(num_bootstraps)]

# Calculate standard deviation of scores
f1_scores, recall_scores, precision_scores = zip(*bootstrap_scores_list)
std_f1_score = np.std(f1_scores)
std_recall_score = np.std(recall_scores)
std_precision_score = np.std(precision_scores)

# Print results
print("Standard Deviation of F1 Scores (Bootstrap):", std_f1_score)
print("Standard Deviation of Recall Scores (Bootstrap):", std_recall_score)
print("Standard Deviation of Precision Scores (Bootstrap):", std_precision_score)
print("f1_score", np.mean(f1_scores))
print("recall", np.mean(recall_scores))
print("precision", np.mean(precision_scores))
print("std_f1_score", std_f1_score)
print("std_recall_score", std_recall_score)
print("std_precision_score", std_precision_score)

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p

Standard Deviation of F1 Scores (Bootstrap): 0.0
Standard Deviation of Recall Scores (Bootstrap): 0.0
Standard Deviation of Precision Scores (Bootstrap): 0.0
f1_score 0.0
recall 0.0
precision 0.0
std_f1_score 0.0
std_recall_score 0.0
std_precision_score 0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p

In [ ]:
with open('autism-test-assessment.npy', 'wb') as f:
    np.save(f, cv_scores)

In [ ]:
#baseline-all
import numpy as np
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
from sklearn.model_selection import cross_validate, train_test_split
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk
!python -m nltk.downloader stopwords


# Load the trained model and vectorizer
model = LogisticRegression(multi_class='multinomial', solver='saga')
vectorizer = TfidfVectorizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

# Function to calculate F1, recall, and precision scores
def bootstrap_scores(true_labels, preds):
    indices = np.random.choice(len(true_labels), len(true_labels), replace=True)
    bootstrap_true_labels = np.take(true_labels, indices)
    bootstrap_preds = np.take(preds, indices)
    f1 = f1_score(bootstrap_true_labels, bootstrap_preds)
    recall = recall_score(bootstrap_true_labels, bootstrap_preds)
    precision = precision_score(bootstrap_true_labels, bootstrap_preds)
    return f1, recall, precision

# Load and preprocess the training data
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))

# Preprocess the text data
X = vectorizer.fit_transform(all_data['selected_posts'])
y = y_true_train


# Define the scoring metrics
scoring = {
    'f1': make_scorer(f1_score),
    'sensitivity': make_scorer(recall_score),
    'specificity': make_scorer(lambda y_true, y_pred: precision_score(y_true, y_pred))
}

# Perform cross-validation
cv_scores = cross_validate(model, X, y, cv=5, scoring=scoring, return_estimator=True, return_indices=True)

# Calculate the mean and standard deviation of the scores
f1_mean = np.mean(cv_scores['test_f1'])
f1_std = np.std(cv_scores['test_f1'])
sensitivity_mean = np.mean(cv_scores['test_sensitivity'])
sensitivity_std = np.std(cv_scores['test_sensitivity'])
specificity_mean = np.mean(cv_scores['test_specificity'])
specificity_std = np.std(cv_scores['test_specificity'])
'''
# Get the predictions for each fold
fold_predictions_cross = []
for estimator, test_index in zip(cv_scores['estimator'], cv_scores['indices']['test']):
    print(test_index)
    X_test_fold = X.tocsr()[test_index]
    y_test_fold = y[test_index]
    y_pred_fold = estimator.predict(X_test_fold)
    fold_predictions_cross.append((y_test_fold, y_pred_fold))
'''
print("Cross-validation scores:")
print(f"F1 Score: {f1_mean:.4f} ± {f1_std:.4f}")
print(f"Sensitivity: {sensitivity_mean:.4f} ± {sensitivity_std:.4f}")
print(f"Specificity: {specificity_mean:.4f} ± {specificity_std:.4f}")

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Cross-validation scores:
F1 Score: 0.1354 ± 0.0524
Sensitivity: 0.0796 ± 0.0286
Specificity: 0.8118 ± 0.3520


In [ ]:
with open('autism-all-simple.npy', 'wb') as f:
    np.save(f, cv_scores)

In [ ]:
#baseline+test
import numpy as np
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
from sklearn.model_selection import cross_validate, train_test_split
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk
!python -m nltk.downloader stopwords

# Load the trained model and vectorizer
model = LogisticRegression(multi_class='multinomial', solver='saga')
vectorizer = TfidfVectorizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

# Function to calculate F1, recall, and precision scores
def bootstrap_scores(true_labels, preds):
    indices = np.random.choice(len(true_labels), len(true_labels), replace=True)
    bootstrap_true_labels = np.take(true_labels, indices)
    bootstrap_preds = np.take(preds, indices)
    f1 = f1_score(bootstrap_true_labels, bootstrap_preds)
    recall = recall_score(bootstrap_true_labels, bootstrap_preds)
    precision = precision_score(bootstrap_true_labels, bootstrap_preds)
    return f1, recall, precision


# Load and preprocess the training data
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))

# Preprocess the text data
X = vectorizer.fit_transform(all_data['selected_posts'])
y = y_true_train

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Number of bootstrap samples
num_bootstraps = 1000

# Generate bootstrap samples and calculate scores
bootstrap_scores_list = [bootstrap_scores(y_test, y_pred) for _ in range(num_bootstraps)]

# Calculate standard deviation of scores
f1_scores, recall_scores, precision_scores = zip(*bootstrap_scores_list)
std_f1_score = np.std(f1_scores)
std_recall_score = np.std(recall_scores)
std_precision_score = np.std(precision_scores)

# Print results
print("Mean F1 Score:", np.mean(f1_scores))
print("Mean Recall Score:", np.mean(recall_scores))
print("Mean Precision Score:", np.mean(precision_scores))
print("Standard Deviation of F1 Scores (Bootstrap):", std_f1_score)
print("Standard Deviation of Recall Scores (Bootstrap):", std_recall_score)
print("Standard Deviation of Precision Scores (Bootstrap):", std_precision_score)

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Mean F1 Score: 0.06989712821367253
Mean Recall Score: 0.036425017886209
Mean Precision Score: 0.998
Standard Deviation of F1 Scores (Bootstrap): 0.027485387266423675
Standard Deviation of Recall Scores (Bootstrap): 0.014806910557304884
Standard Deviation of Precision Scores (Bootstrap): 0.044676615807377355


In [ ]:
with open('autism-test-simple.npy', 'wb') as f:
    np.save(f, y_pred)